In [ ]:
import os
import shutil
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image

In [ ]:
# Define the dataset class
# Function to list all image files in a directory
def list_files(dir):
    file_list = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(".png"):
                file_list.append(os.path.join(root, file))
    return file_list

In [ ]:
# Directory containing folders where each folder represents a class or label
data_dir = '/content/drive/MyDrive/DATA'

# Directory  to save the split data
output_dir = '/content/drive/MyDrive/DATA'

# Percentage of data to use for training and validation (rest will be used for testing)
train_val_split = 0.8
val_split = 0.1

# Iterate through the folders (classes) in the data directory
for class_dir in os.listdir(data_dir):
    # Construct the full path to the class directory
    class_path = os.path.join(data_dir, class_dir)

    # Skip if class_path is not a directory
    if not os.path.isdir(class_path):
        continue

    print(f"Processing {class_dir} class...")

    # List all the PNG files in the class directory (including subdirectories)
    files = list_files(class_path)

    # Shuffle the files randomly
    random.shuffle(files)

    # Calculate the number of files for training, validation, and testing
    num_train_val_files = int(train_val_split * len(files))
    num_val_files = int(val_split * num_train_val_files)

    train_val_files = files[:num_train_val_files]
    train_files = train_val_files[:-num_val_files]
    val_files = train_val_files[-num_val_files:]
    test_files = files[num_train_val_files:]

    # Create corresponding directories in the output directory
    train_class_dir = os.path.join(output_dir, 'train', class_dir)
    val_class_dir = os.path.join(output_dir, 'val', class_dir)
    test_class_dir = os.path.join(output_dir, 'test', class_dir)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Copy training files to the train directory
    for file_path in train_files:
        file_name = os.path.basename(file_path)
        dst = os.path.join(train_class_dir, file_name)
        shutil.copy(file_path, dst)

    # Copy validation files to the validation directory
    for file_path in val_files:
        file_name = os.path.basename(file_path)
        dst = os.path.join(val_class_dir, file_name)
        shutil.copy(file_path, dst)

    # Copy testing files to the test directory
    for file_path in test_files:
        file_name = os.path.basename(file_path)
        dst = os.path.join(test_class_dir, file_name)
        shutil.copy(file_path, dst)

print("Data split completed successfully.")

Processing 54 class...
Processing 5 class...
Processing 3 class...
Processing 55 class...
Processing 56 class...
Processing 35 class...
Processing 57 class...
Processing 7 class...
Processing 4 class...
Processing 30 class...
Processing 0 class...
Processing 26 class...
Processing 16 class...
Processing 11 class...
Processing 24 class...
Processing 10 class...
Processing 28 class...
Processing 17 class...
Processing 14 class...
Processing 12 class...
Data split completed successfully.


In [ ]:
dataset_root = '/content/drive/MyDrive/DATA'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
class RoadSignDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform

        # Get the list of class folders (labels)
        self.class_names = sorted(os.listdir(self.root_dir))

        # Initialize lists to store image paths and corresponding labels
        self.image_paths = []
        self.labels = []

        # Iterate over class folders
        for label_idx, class_name in enumerate(self.class_names):
            class_dir = os.path.join(self.root_dir, class_name)

            # Get list of image files in the class folder
            image_files = [f for f in os.listdir(class_dir) if f.endswith('.png')]

            # Append image paths and corresponding labels
            self.image_paths.extend([os.path.join(class_dir, img) for img in image_files])
            self.labels.extend([label_idx] * len(image_files))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Define dataset and data loaders for train and validation sets
train_dataset = RoadSignDataset(root_dir=dataset_root, split='train', transform=transform)
test_dataset = RoadSignDataset(root_dir=dataset_root, split='test', transform=transform)
val_dataset = RoadSignDataset(root_dir=dataset_root, split='val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Define the ResNet model
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.class_names))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Move model to device if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Training and evaluation
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            running_accuracy += (predicted == labels).float().mean().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = running_accuracy / len(train_loader)

        # Validation
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                val_loss = criterion(outputs, labels)
                val_running_loss += val_loss.item()

                _, predicted = torch.max(outputs, 1)
                val_running_accuracy += (predicted == labels).float().mean().item()

        val_loss = val_running_loss / len(val_loader)
        val_accuracy = val_running_accuracy / len(val_loader)

        print(f'Epoch {epoch + 1}/{num_epochs}, '
              f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, '
              f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

    print('Training completed')

In [ ]:
# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader)

# Evaluate the model on test set
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Evaluate the model
evaluate_model(model, test_loader)


Epoch 1/25, Training Loss: 2.5684, Training Accuracy: 23.81%, Validation Loss: 2.2864, Validation Accuracy: 31.25%
Epoch 2/25, Training Loss: 1.8294, Training Accuracy: 44.54%, Validation Loss: 1.7290, Validation Accuracy: 46.09%
Epoch 3/25, Training Loss: 1.4295, Training Accuracy: 56.16%, Validation Loss: 1.5546, Validation Accuracy: 49.36%
Epoch 4/25, Training Loss: 1.1975, Training Accuracy: 62.02%, Validation Loss: 1.3619, Validation Accuracy: 57.96%
Epoch 5/25, Training Loss: 1.0431, Training Accuracy: 67.65%, Validation Loss: 1.2177, Validation Accuracy: 62.13%
Epoch 6/25, Training Loss: 0.8987, Training Accuracy: 73.11%, Validation Loss: 1.1664, Validation Accuracy: 63.45%
Epoch 7/25, Training Loss: 0.7960, Training Accuracy: 75.13%, Validation Loss: 1.0630, Validation Accuracy: 68.94%
Epoch 8/25, Training Loss: 0.6927, Training Accuracy: 81.82%, Validation Loss: 0.9870, Validation Accuracy: 75.19%
Epoch 9/25, Training Loss: 0.5709, Training Accuracy: 85.74%, Validation Loss: 1